In [1]:
import numpy as np
import pandas as pd
import kagglehub
import sklearn
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [49]:
df_test = pd.read_csv("test.csv")

In [50]:
df_train = pd.read_csv("train.csv")

In [51]:
df = pd.concat([df_test, df_train])

In [52]:
df_test.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,53,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,mon,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.970,5228.1,no
1,52,services,divorced,high.school,no,no,no,telephone,may,mon,...,3,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,29,technician,single,university.degree,no,no,no,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0,no
3,26,housemaid,married,basic.4y,no,no,no,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
4,49,management,divorced,university.degree,no,no,no,telephone,jun,thu,...,2,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1,no
5,30,services,married,high.school,no,yes,no,telephone,jun,fri,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
6,38,blue-collar,married,basic.9y,no,yes,no,cellular,jul,wed,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
7,43,blue-collar,married,basic.9y,unknown,no,no,telephone,may,fri,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.864,5191.0,yes
8,38,admin.,married,university.degree,no,no,no,telephone,may,tue,...,3,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,41,technician,married,professional.course,no,no,no,cellular,aug,mon,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,no


In [53]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6590 entries, 0 to 6589
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             6590 non-null   int64  
 1   job             6590 non-null   object 
 2   marital         6590 non-null   object 
 3   education       6590 non-null   object 
 4   default         6590 non-null   object 
 5   housing         6590 non-null   object 
 6   loan            6590 non-null   object 
 7   contact         6590 non-null   object 
 8   month           6590 non-null   object 
 9   day_of_week     6590 non-null   object 
 10  duration        6590 non-null   int64  
 11  campaign        6590 non-null   int64  
 12  pdays           6590 non-null   int64  
 13  previous        6590 non-null   int64  
 14  poutcome        6590 non-null   object 
 15  emp.var.rate    6590 non-null   float64
 16  cons.price.idx  6590 non-null   float64
 17  cons.conf.idx   6590 non-null   f

In [54]:
df.drop(columns = ["job", "marital", "education", "contact", "month", "day_of_week"], inplace = True)

In [55]:
df.drop(columns = ["default", "poutcome"], inplace = True)

In [56]:
df_test.head(20)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,53,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,mon,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.970,5228.1,no
1,52,services,divorced,high.school,no,no,no,telephone,may,mon,...,3,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,29,technician,single,university.degree,no,no,no,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0,no
3,26,housemaid,married,basic.4y,no,no,no,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
4,49,management,divorced,university.degree,no,no,no,telephone,jun,thu,...,2,999,0,nonexistent,1.4,94.465,-41.8,4.961,5228.1,no
5,30,services,married,high.school,no,yes,no,telephone,jun,fri,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
6,38,blue-collar,married,basic.9y,no,yes,no,cellular,jul,wed,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
7,43,blue-collar,married,basic.9y,unknown,no,no,telephone,may,fri,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.864,5191.0,yes
8,38,admin.,married,university.degree,no,no,no,telephone,may,tue,...,3,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,41,technician,married,professional.course,no,no,no,cellular,aug,mon,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,no


In [57]:
df_test.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [58]:
df['y'] = df['y'].map({'yes': 1, 'no': 0})
df['housing'] = df['housing'].map({'yes': 1, 'no': 0})
df['loan'] = df['loan'].map({'yes': 1, 'no': 0})

In [59]:
X = df.drop(columns=['y'])
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Split lagi untuk validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.25,  # 60/20/20 total
    random_state=42,
    stratify=y_train
)

print("Train:", X_train.shape)
print("Validation:", X_val.shape)
print("Test:", X_test.shape)

Train: (19770, 12)
Validation: (6590, 12)
Test: (6590, 12)


In [60]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

# Konversi ke DataFrame agar bisa dilacak kolomnya
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled   = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_scaled  = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [61]:
df.head()

,age,housing,loan,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,53,1.0,0.0,144,1,999,0,1.4,93.444,-36.1,4.970,5228.1,0
1,52,0.0,0.0,142,3,999,0,1.1,93.994,-36.4,4.857,5191.0,0
2,29,0.0,0.0,171,1,999,0,1.1,93.994,-36.4,4.858,5191.0,0
3,26,0.0,0.0,695,1,999,0,1.4,93.918,-42.7,4.958,5228.1,0
4,49,0.0,0.0,163,2,999,0,1.4,94.465,-41.8,4.961,5228.1,0


In [62]:
param_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 9, None],
    'min_samples_split': [2, 5, 10]
}

grid_dt = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_dt,
    cv=5,
    scoring='accuracy'
)
grid_dt.fit(X_train_scaled, y_train)
best_dt = grid_dt.best_estimator_

y_val_pred_dt = best_dt.predict(X_val_scaled)
acc_dt = accuracy_score(y_val, y_val_pred_dt)
prec_dt = precision_score(y_val, y_val_pred_dt)
rec_dt = recall_score(y_val, y_val_pred_dt)
f1_dt = f1_score(y_val, y_val_pred_dt)

print("=== Decision Tree ===")
print(grid_dt.best_params_, f"| Acc: {acc_dt:.4f}, F1: {f1_dt:.4f}")

# --- Random Forest ---
param_rf = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'criterion': ['gini', 'entropy']
}

grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_rf,
    cv=5,
    scoring='accuracy'
)
grid_rf.fit(X_train_scaled, y_train)
best_rf = grid_rf.best_estimator_

y_val_pred_rf = best_rf.predict(X_val_scaled)
acc_rf = accuracy_score(y_val, y_val_pred_rf)
prec_rf = precision_score(y_val, y_val_pred_rf)
rec_rf = recall_score(y_val, y_val_pred_rf)
f1_rf = f1_score(y_val, y_val_pred_rf)

print("=== Random Forest ===")
print(grid_rf.best_params_, f"| Acc: {acc_rf:.4f}, F1: {f1_rf:.4f}")

# --- XGBoost ---
param_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_xgb = GridSearchCV(
    xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
    param_xgb,
    cv=5,
    scoring='accuracy'
)
grid_xgb.fit(X_train_scaled, y_train)
best_xgb = grid_xgb.best_estimator_

y_val_pred_xgb = best_xgb.predict(X_val_scaled)
acc_xgb = accuracy_score(y_val, y_val_pred_xgb)
prec_xgb = precision_score(y_val, y_val_pred_xgb)
rec_xgb = recall_score(y_val, y_val_pred_xgb)
f1_xgb = f1_score(y_val, y_val_pred_xgb)

print("=== XGBoost ===")
print(grid_xgb.best_params_, f"| Acc: {acc_xgb:.4f}, F1: {f1_xgb:.4f}")

=== Decision Tree ===
{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 2} | Acc: 0.9083, F1: 0.5661
=== Random Forest ===
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100} | Acc: 0.9111, F1: 0.5067
=== XGBoost ===
{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8} | Acc: 0.9102, F1: 0.5508


In [63]:
results = pd.DataFrame({
    'Model': ['Decision Tree', 'Random Forest', 'XGBoost'],
    'Accuracy': [acc_dt, acc_rf, acc_xgb],
    'Precision': [prec_dt, prec_rf, prec_xgb],
    'Recall': [rec_dt, rec_rf, rec_xgb],
    'F1-Score': [f1_dt, f1_rf, f1_xgb]
})

print("\n=== Perbandingan Model ===")
print(results.sort_values(by='Accuracy', ascending=False))


=== Perbandingan Model ===
           Model  Accuracy  Precision    Recall  F1-Score
1  Random Forest  0.911077   0.677928  0.404570  0.506734
2        XGBoost  0.910167   0.632404  0.487903  0.550835
0  Decision Tree  0.908346   0.608025  0.529570  0.566092


In [78]:
X_final_train = np.concatenate((X_train_scaled, X_val_scaled), axis=0)
y_final_train = np.concatenate((y_train, y_val), axis=0)

final_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42,
    class_weight='balanced'
)

final_model.fit(X_final_train, y_final_train)

y_test_pred = final_model.predict(X_test_scaled)

acc = accuracy_score(y_test, y_test_pred)
prec = precision_score(y_test, y_test_pred)
rec = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print("\n=== Evaluation (Test Set) ===")
print(f"Akurasi   : {acc:.4f}")
print(f"Precision : {prec:.4f}")
print(f"Recall    : {rec:.4f}")
print(f"F1-Score  : {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


=== Evaluation (Test Set) ===
Akurasi   : 0.8329
Precision : 0.3975
Recall    : 0.9301
F1-Score  : 0.5569

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.82      0.90      5846
           1       0.40      0.93      0.56       744

    accuracy                           0.83      6590
   macro avg       0.69      0.88      0.73      6590
weighted avg       0.92      0.83      0.86      6590



C:\Users\barak\anaconda3\envs\scatest\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [75]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

# Parameter training XGBoost
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'seed': 42,
}
num_boost_round = 300

# Latih model
bst = xgb.train(params, dtest, num_boost_round=num_boost_round)

# Prediksi probabilitas dan hitung akurasi dengan threshold 0.5
y_prob = bst.predict(dtest)
y_pred = (y_prob >= 0.5).astype(int)
print(f"Final Score XGBOOST? Akurasi: {accuracy_score(y_test, y_pred):.4f}")

Final Score XGBOOST? Akurasi: 0.9791


In [80]:
clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(X_test, y_test)

y_pred = clf.predict(X_test)

print(f"Final Scrore Decision Tree? Akurasi: {accuracy_score(y_test, y_pred)}")

Final Scrore Decision Tree? Akurasi: 0.9110773899848255
